In [2]:
# install packages in libs (as editable)
!python -m pip install -e ../libs

70.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Obtaining file:///Users/n0c09jf/code/github/binsense/libs
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for binsense-libs (pyproject.toml) ... done
  Created wheel for binsense-libs: filename=binsense_libs-0.0.1-py2.py3-none-any.whl size=1137 sha256=8524b93cea3793ac8f93136a46b3b775f211868867a04c7e319690205c497b92
  Stored in directory: /private/var/folders/vm/2g2pvy116zzglvllmm0n_f100000gx/T/pip-ephem-wheel-cache-ibxzsdqi/wheels/77/72/16/1e76e1dcd0e567ae2a1096c6d731674e5dee84ed8ab15b770d
Successfully built binsense-libs
  Attempting uninstall: binsense-libs
    Found existing installation: binsense-libs 0.0.1
    Uninstalling binsense-libs-0.0.1:
      Successfully uninstalled binsense-libs-0.0.1


In [1]:
from binsense.config import DATA_DIR
print(DATA_DIR)

import binsense.downloader as d
d.download()

import binsense.metadata as m
bin_df, item_df = m.load()
print(bin_df.shape, item_df.shape)


/Users/n0c09jf/code/github/binsense/_data


loading bin-metadata: 100%|██████████| 3875/3875 [00:43<00:00, 90.09it/s] 

(3875, 6) (10144, 12)


In [4]:
from huggingface_hub import hf_hub_download
from safetensors import safe_open

import json, torch

cfg_file_path = hf_hub_download(repo_id="google/owlv2-base-patch16-ensemble", filename="config.json")
print('config.json', cfg_file_path)

model_file_path = hf_hub_download(repo_id="google/owlv2-base-patch16-ensemble", filename="model.safetensors")
print('model.safetensors', model_file_path)

preproc_file_path = hf_hub_download(repo_id="google/owlv2-base-patch16-ensemble", filename="preprocessor_config.json")
print('preprocessor_config.json', preproc_file_path)

state_dict = {}
with safe_open(model_file_path, framework="pt", device="cpu") as f:
   for key in f.keys():
       state_dict[key] = f.get_tensor(key)
print(state_dict.keys())

with open(cfg_file_path, 'r') as f:
    config = json.load(f)
print(json.dumps(config, indent=2))

config.json /Users/n0c09jf/.cache/huggingface/hub/models--google--owlv2-base-patch16-ensemble/snapshots/7c77fc1316a353da3109f81d80ae118dd58fd55b/config.json
model.safetensors /Users/n0c09jf/.cache/huggingface/hub/models--google--owlv2-base-patch16-ensemble/snapshots/7c77fc1316a353da3109f81d80ae118dd58fd55b/model.safetensors
dict_keys(['box_head.dense0.bias', 'box_head.dense0.weight', 'box_head.dense1.bias', 'box_head.dense1.weight', 'box_head.dense2.bias', 'box_head.dense2.weight', 'class_head.dense0.bias', 'class_head.dense0.weight', 'class_head.logit_scale.bias', 'class_head.logit_scale.weight', 'class_head.logit_shift.bias', 'class_head.logit_shift.weight', 'layer_norm.bias', 'layer_norm.weight', 'objectness_head.dense0.bias', 'objectness_head.dense0.weight', 'objectness_head.dense1.bias', 'objectness_head.dense1.weight', 'objectness_head.dense2.bias', 'objectness_head.dense2.weight', 'owlv2.logit_scale', 'owlv2.text_model.embeddings.position_embedding.weight', 'owlv2.text_model.emb